In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle

#RENSET DATASETT: STOPWORDS ETC:
#inp = pd.read_csv('Input/Cleaned_besk_eform.csv', dtype=object, encoding='iso-8859-1', names=['besk'], skiprows=[0]) 

#nace=inp["besk"].str.split(";", n=1, expand=True)

#nace=nace[nace[0].notnull()]
#nace=shuffle(nace)


#from sklearn.model_selection import train_test_split

#besk=nace[0].values
#sn07=nace[1].values
###############################

#URENSET DATASETT:
nace = pd.read_csv('Input/BESK_ALLE.csv', sep=';', encoding='iso-8859-1') 

nace=nace[nace["besk"].notnull()]
nace=shuffle(nace)

"""Split into training and test set"""

from sklearn.model_selection import train_test_split

besk=nace["besk"].values
sn07=nace["SN07_1"].values
###############################

#PRETRAINED VECTOR
import io
import numpy as np
file = io.open('Input/cc.no.300.vec', 'r', encoding='utf-8', newline='\n', errors='ignore')
vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
  values = line.split()
  word = values [0]
  vector = np.asarray(values[1:], dtype='float32')
  vocab_and_vectors[word] = vector
###############################

from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_encode = lab_enc.fit_transform(sn07)

import keras

y = keras.utils.to_categorical(y_encode, 841)


from keras.preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(besk)
X=t.texts_to_sequences(besk)

vocab_size=len(t.word_index)+1

word_index = t.word_index

from keras.preprocessing.sequence import pad_sequences

maxlen=162

X_pad=pad_sequences(X, padding='post', maxlen=maxlen)


X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.5, random_state=1000)

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
  embedding_vector = vocab_and_vectors.get(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    
#normalizer = preprocessing.Normalizer().fit(X_train)
#X_train_norm=normalizer.transform(X_train)
#X_test_norm=normalizer.transform(X_test)

#x_train= tf.keras.utils.normalize(X_train, axis=1)
#x_test= tf.keras.utils.normalize(X_test, axis=1)


#longest_string_sn07=max(sn07, key=len)
#len(longest_string_sn07)
#longest_string_besk=max(besk, key=len)
#len(longest_string_besk)
#max(y_encode)





In [11]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, SeparableConv1D, MaxPooling1D
from keras.optimizers import SGD, Adam
from tensorflow.keras.utils import normalize
import tensorflow as tf
import keras
import time
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import LearningRateScheduler
import math

# learning rate schedule
#def step_decay(epoch):
#    initial_lrate = 0.001
#    drop = 0.1
#    epochs_drop = 2
#    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#    return lrate

#input_dim = X_train.shape[1:]
embedding_dim=300

#NAME="{}-dense-{}-conv-{}-layer-{}".format(dense_layer, conv_layer, layer_size, int(time.time()))
#tensorboard=TensorBoard(log_dir='logs/7/{}'.format(NAME))
#print(NAME)
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim,
                           input_length=maxlen, 
                           weights=[embedding_matrix],
                           trainable=True))
model.add(layers.SeparableConv1D(256 ,5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))

model.add(layers.Flatten())
            
model.add(layers.Dropout(0.2))

model.add(layers.Dense(841, activation='softmax'))
epochs = 10
learning_rate = 0.001
decay_rate = learning_rate / epochs
adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#lrate = LearningRateScheduler(step_decay)
#callbacks_list = [tensorboard]
model.fit(X_train, y_train,
        epochs=10,
        validation_data=(X_test, y_test))#, callbacks=callbacks_list)

#åpne tensorboard: tensorboard --logdi=ml/logs/
#Accuracy på et utvalg fra datasettet: 52% for renset og urenset sett
#Accuracy for hele settet: 60% for renset og urenset sett

Train on 790937 samples, validate on 790938 samples
Epoch 1/10
790937/790937 [==============================] - 16588s 21ms/step - loss: 2.1527 - acc: 0.5474 - val_loss: 1.8115 - val_acc: 0.5971
Epoch 2/10
790937/790937 [==============================] - 16558s 21ms/step - loss: 1.5774 - acc: 0.6336 - val_loss: 1.7559 - val_acc: 0.6059
Epoch 3/10
790937/790937 [==============================] - 16549s 21ms/step - loss: 1.4019 - acc: 0.6684 - val_loss: 1.7676 - val_acc: 0.6069
Epoch 4/10
790937/790937 [==============================] - 16666s 21ms/step - loss: 1.2942 - acc: 0.6923 - val_loss: 1.7876 - val_acc: 0.6057
Epoch 5/10
790937/790937 [==============================] - 16626s 21ms/step - loss: 1.2182 - acc: 0.7099 - val_loss: 1.8090 - val_acc: 0.6043
Epoch 6/10
790937/790937 [==============================] - 17696s 22ms/step - loss: 1.1615 - acc: 0.7232 - val_loss: 1.8311 - val_acc: 0.6033
Epoch 7/10
790937/790937 [==============================] - 16675s 21ms/step - loss: 1.117